In [1]:
import requests
import googlemaps
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
from pyproj import Geod
from ast import literal_eval
import keyring
import json

C:\Users\yianz\AppData\Local\Temp\ipykernel_21280\1920686190.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
API_KEY = keyring.get_password("goomaps", "yianzhang14")

In [3]:
gmaps = googlemaps.Client(key=API_KEY)

In [4]:
# somehwere in manhattan
loc = (40.73188892751728, -73.98927188478527)

In [5]:
df = pd.read_csv("data/urgent_cleaned.csv")
df

,Unnamed: 0,full address,NPI,Site Name,Address,Room/Suite,Town/City,CountyFIPS,County Name,State,...,Servicing County 4,Servicing County 5,OASAS Program Reporting Unit Number,OASAS Provider Number,OMH ID,Plan Name,Provider Network Data System Identification Number,Quarter/Year,geometry,type
0,2,"1 BROOKDALE PLZ RM LL91, BROOKLYN, NY 11212",1548217763,FLUSHING HOSPITAL MEDICAL CENTER,1 BROOKDALE PLZ,RM LL91,BROOKLYN,36047,Kings,NY,...,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (307273.1349615707 54330.81319598604),NPI-2
1,4,"1 FORDHAM PLZ, BRONX, NY 10458",1720414154,MONTEFIORE NEW ROCHELLE HOSPITAL,1 FORDHAM PLZ,NaN,BRONX,36005,Bronx,NY,...,999,999,NaN,NaN,9999999999,Consolidated Health Plans: Cigna,168,Quarter 1 2021,POINT (309341.77659211366 77031.53626176476),NPI-2
2,6,"1 GUSTAVE L LEVY PL, NEW YORK, NY 10029",1013948595,MOUNT SINAI HOSPITAL,1 GUSTAVE L LEVY PL,NaN,NEW YORK,36061,New York,NY,...,999,999,1.000000e+10,1.000000e+10,6399020,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (303913.30307026475 69277.7115768047),NPI-2
3,7,"1 PROSPECT PARK W, BROOKLYN, NY 11215",1043251390,PARK SLOPE EMERGENCY PHYS SVC,1 PROSPECT PARK W,NaN,BROOKLYN,36047,Kings,NY,...,999,999,NaN,NaN,NaN,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (302481.54769885784 56164.37563754736),NPI-2
4,10,"100 E 77TH ST, NEW YORK, NY 10075",1316147176,LENOX HILL HOSPITAL,100 E 77TH ST,NaN,NEW YORK,36061,New York,NY,...,999,999,NaN,NaN,6649020,Jefferson-Lewis Et. Al. School Employees’ Heal...,212,Quarter 1 2021,POINT (303316.91374236497 67426.23223508784),NPI-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,856,"9777 QUEENS BLVD, REGO PARK, NY 11374",1437230034,NYU HOSPITALS CENTER - CANCER CENTER,9777 QUEENS BLVD,NaN,REGO PARK,36081,Queens,NY,...,999,999,NaN,NaN,NaN,New York Quality Healthcare Corporation,44,Quarter 1 2021,POINT (311960.8099653642 62459.85496309886),NPI-2
562,857,"9920 4TH AVE STE 205, BROOKLYN, NY 11209",1205868528,SUNY DOWNSTATE MED CENTER,9920 4TH AVE,STE 205,BROOKLYN,36047,Kings,NY,...,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (297167.59793859185 49596.59393249073),NPI-2
563,858,"JFK ARPT 14-140 TENANT #4 BLD, JAMAICA, NY 11430",1417349507,METRO URGENT MEDICAL CARE OF QUEENS,JFK ARPT 14-140 TENANT #4 BLD,NaN,JAMAICA,36081,Queens,NY,...,999,999,NaN,NaN,NaN,"Oxford Health Plans (NY), Inc.: Metro",143,Quarter 1 2021,POINT (316830.18732158607 54648.54333367746),NPI-2
564,860,"MADISON & 100 ST, NEW YORK, NY 10029",1255375986,MOUNT SINAI ELMHURST FCLTY PRAC G,MADISON & 100 ST,NaN,NEW YORK,36061,New York,NY,...,999,999,NaN,NaN,NaN,"UnitedHealthcare of New York, Inc.",148,Quarter 1 2021,POINT (304747.59900113015 69812.57038312979),NPI-2


In [6]:
def get_latlon(address):
    latlon = gmaps.geocode(address)[0]
    return ', '.join(str(x) for x in latlon["geometry"]["location"].values())

In [7]:
df["latlon"] = df["full address"].apply(get_latlon)

In [67]:
def get_place_details(row):
    res = {}
    res["status"] = "ok"
    res["opening_hours"] = ""
    res["place_id"] = ""
    
    place_id = gmaps.places(query=row["Site Name"] + " " + row["full address"], location=row["latlon"], radius=64)
    if place_id["status"] != "OK":
        res["gmaps_status"] = "error - can't find place id"
        print("can't find place id", row["Site Name"])
        return res
    
    
    place = gmaps.place(place_id["results"][0]["place_id"])
    if place["status"] != "OK":
        print("can't match place id", row["Site Name"])
        res["gmaps_status"] = "error - can't match place id"
        return res
    place = place["result"]
    if "opening_hours" in place:
        res["opening_hours"] = json.dumps(place["opening_hours"])
    if "place_id" in place:
        res["place_id"] = place["place_id"]
        
    return res

In [68]:
applied_df = df.apply(lambda x: get_place_details(x), axis=1, result_type="expand")

can't find place id FLUSHING HOSPITAL MEDICAL CENTER
can't find place id 210 MJD MED PLLC DBA SILVER STAR UC
can't find place id FLUSHING HOSPITAL MEDICAL CENTER
can't find place id MMC EMERGENCY PHYSICIANS FPP
can't find place id NYU LANGONE MEDHATTAN
can't find place id MOUNT SINAI ELMHURST FCLTY PRAC G
can't find place id PENINSULA MEDICAL CARE PLLC
can't find place id MOUNT SINAI ELMHURST FCLTY PRAC G
can't find place id WMC CORNELL EMERGENCY PHYSICIANS
can't find place id TAKE CARE MED HLTH NY PC
can't find place id ROCHESTER GENERAL HOSP-PROF FEES
can't find place id MOUNT SINAI ELMHURST FCLTY PRAC G
can't find place id CITY MEDICAL OF UPPER EAST SIDE, PLLC DBA CITYMD U
can't find place id MOONLIGHT PEDIATRICS
can't find place id ESSEN MEDICAL URGICARE PLLC DBA METRO URGICARE PLL
can't find place id WMC CORNELL EMERGENCY PHYSICIANS
can't find place id BRONX LEBANON HSP/EM DEPT
can't find place id KINGSBROOK MEDICAL ASSOCIATES
can't find place id ESSEN MEDICAL URGICARE PLLC DBA ME

In [69]:
applied_df

,status,opening_hours,place_id,gmaps_status
0,ok,,,error - can't find place id
1,ok,"{""open_now"": true, ""periods"": [{""close"": {""day...",ChIJK5Kt9X_0wokRtSbTJb_VT5U,NaN
2,ok,,ChIJUyrmBQ9ZwokRJFt7Ck3MO1g,NaN
3,ok,,ChIJJ45eeghbwokRhTIYFdgy_tY,NaN
4,ok,"{""open_now"": true, ""periods"": [{""open"": {""day""...",ChIJra13dJVYwokRh3kQ5-p7Nio,NaN
...,...,...,...,...
561,ok,,ChIJ_4cGwTBewokR_-LKjhQKw2Y,NaN
562,ok,"{""open_now"": true, ""periods"": [{""open"": {""day""...",ChIJSaTDdllFwokR7WiY-lXXlBQ,NaN
563,ok,"{""open_now"": true, ""periods"": [{""close"": {""day...",ChIJk_ZCRAhnwokRPXEE2Va4IQE,NaN
564,ok,,ChIJT1R9W6dYwokRleZSU1oGeCA,NaN


In [70]:
df_combined = pd.concat([df, applied_df], axis=1)

In [71]:
df_combined.to_csv("data/urgent_time.csv")

In [72]:
(df_combined["opening_hours"] == "").sum()

186

In [73]:
gmaps.place("ChIJK5Kt9X_0wokRtSbTJb_VT5U")

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '1',
    'short_name': '1',
    'types': ['street_number']},
   {'long_name': 'Fordham Plaza',
    'short_name': 'Fordham Plaza',
    'types': ['route']},
   {'long_name': 'Belmont',
    'short_name': 'Belmont',
    'types': ['neighborhood', 'political']},
   {'long_name': 'The Bronx',
    'short_name': 'Bronx',
    'types': ['sublocality_level_1', 'sublocality', 'political']},
   {'long_name': 'Bronx County',
    'short_name': 'Bronx County',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'New York',
    'short_name': 'NY',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'United States',
    'short_name': 'US',
    'types': ['country', 'political']},
   {'long_name': '10458', 'short_name': '10458', 'types': ['postal_code']},
   {'long_name': '5871',
    'short_name': '5871',
    'types': ['postal_code_suffix']}],
  'adr_address': '<span class="street-ad